In [9]:
from flask import Flask, render_template,jsonify,request
import pandas as pd
import plotly.express as px
import os

app = Flask(__name__)

# sample dataframes
df1 = pd.DataFrame({
    'Nombre': ['Reservas0', 'Reservas1', 'Reservas2'],
    'Descripción': ["LOREM IPSUM", "LOREM IPSUM", "LOREM IPSUM"],
    'Staff': [3, 5, 2],
    'Senior': [3, 3, 0],
    'Manager': [1, 1, 1],
    'Senior Manager': [1,0, 0],
    'Socio': [0, 1, 0]
})

df2 = pd.DataFrame({
    'Staff': [1],
    'Senior': [3],
    'Manager': [5],
    'Senior Manager': [10],
    'Socio': [15]
})
df3 = pd.DataFrame({
    'Costo': [350]
})

df4=pd.concat([df2.multiply(df1.loc[x]).fillna(0)*int(df3.loc[0]) for x in range(df1.shape[0])]).reset_index(drop=True)
print(df4)
df4.loc[:,('Descripción','Nombre')]=df1.loc[:,('Descripción','Nombre')]

dataframes = [df1, df2,df3,df4]
titles = ['Servicios', 'Rangos',"Costo staff","Presupuesto"]

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', title='Inicio')


@app.route('/proyectos')
def proyectos():
    return render_template('proyectos.html', dataframes=dataframes,titles=titles)
# Endpoint for updating the DataFrames


@app.route('/update_cell', methods=['POST'])
def update_dataframe():
    data = request.get_json()
    print('data ',data)
    #{'table_index': 0, 'row_index': 1, 'column_index': 0, 'value': 'Bob'}
    row_index = int(data['row_index'])
    column_name = data['column_index']
    new_value = data['value']
    df_index = int(data['table_index'])
    #dataframes[df_index].iloc[row_index, column_name] = int(new_value)
    #print(dataframes[df_index])
    ##########
    #merged_df = pd.concat(dataframes)
    #merged_df.columns = merged_df.columns.astype(str)
    #merged_df.to_parquet('data.parquet', engine='auto', compression='snappy')
    # guardar cada df como csv
    #return jsonify({'status': 'success'})
    df = dataframes[df_index]
    
    if row_index >= df.shape[0]:
        # Add a new row to the DataFrame if updating a row that doesn't exist
        new_row = pd.Series({'Nombre': '', 'Descripción': '', 'Staff': 0, 'Senior': 0, 'Manager': 0, 'Senior Manager': 0, 'Socio': 0})
        df = df.append(new_row, ignore_index=True)

    # Update the value in the DataFrame
    df.iloc[row_index, column_name] = int(new_value)
    dataframes[df_index] = df
    
    # Send the updated DataFrame to the frontend
    response_data = df.to_json(orient='records')
    return jsonify({'status': 'success', 'data': response_data})


@app.route('/recalculate_df4', methods=['POST'])
def recalculate_df4():
    global dataframes
#    df4 = pd.concat([df2.multiply(df1.loc[x]).fillna(0)*int(df3.loc[0]) for x in range(df1.shape[0])]).reset_index()
#    df4.loc[:,('Descripción','Nombre')]=df1.loc[:,('Descripción','Nombre')]
    #df4.columns=df1.columns
#    response_data = df4.to_json(orient='records')
#    print(df4)
   # multi=dataframes[0][['Staff','Senior','Manager','Senior Manager','Socio']].astype(int)
    
    dataframes[3]=pd.concat([dataframes[1].multiply(dataframes[0].iloc[x]).fillna(0)*int(dataframes[2].loc[0]) for x in range(dataframes[0].shape[0])]).reset_index(drop=True)
    print(dataframes[0].loc[:,('Descripción','Nombre')])
    #dataframes[3].loc[:,('Descripción','Nombre')]=dataframes[0].loc[:,('Descripción','Nombre')]
    #df4.columns=df1.columns


    response_data = dataframes[3].to_json(orient='records')
    print(dataframes[3])
    return jsonify({'status': 'success', 'data': response_data})

@app.route('/update_df1', methods=['POST'])
def update_df1():
    data = request.get_json()
    data_dict = {
    'Nombre': [str(data[-1][0])],
    'Descripción': [str(data[-1][1])],
    'Staff': [int(data[-1][2])],
    'Senior': [int(data[-1][3])],
    'Manager': [int(data[-1][4])],
    'Senior Manager': [int(data[-1][5])],
    'Socio': [int(data[-1][6])]
    }
    
    new_row = pd.DataFrame(data_dict)
    
    dataframes[3] = pd.concat([dataframes[3], new_row], axis=0)
    print(dataframes[3])
    dataframes[3] = dataframes[3].reset_index(drop=True)

    temp_nombre_descripcion = dataframes[0][['Nombre', 'Descripción']]
    dataframes[0] = pd.concat([dataframes[0], new_row], axis=0)
    dataframes[3][['Nombre', 'Descripción']] = temp_nombre_descripcion

    response_data = dataframes[3].to_json(orient='records')
    
    return jsonify({'status': 'success', 'data': response_data})

@app.route('/save_dataframes', methods=['GET'])
def save_dataframes():
    # Create a directory to store the CSV files if it doesn't exist
    if not os.path.exists('dataframes'):
        os.makedirs('dataframes')
    
    # Save each dataframe to a separate CSV file
    for i, df in enumerate(dataframes):
        filename = f'dataframes/df{i+1}.csv'
        df.to_csv(filename, index=False)
    
    return jsonify({'status': 'success'})

if __name__ == '__main__':
    app.run(debug=False)


   Descripción  Manager  Nombre  Senior  Senior Manager  Socio  Staff
0            0     1750       0    3150            3500      0   1050
1            0     1750       0    3150               0   5250   1750
2            0     1750       0       0               0      0    700
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Mar/2023 22:37:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2023 22:37:09] "GET /proyectos HTTP/1.1" 200 -
